In [1]:
import module.db_module as db_module
db_class = db_module.Database() #db연결 생성

In [2]:
def xstr(s):
    if s is None:
        return ''
    return str(s)

In [3]:
import pandas as pd
import numpy as np
import re
import regex
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
pd.set_option('display.max_rows', None) ## 모든 열을 출력한다.

phone_list = pd.read_sql_table('g5_phone_list',db_class.engine_conn)

# AKA 테이블

In [4]:
aka_table = {
    "mintit":{
        0:{
            "column":"mintit_key",
            "key":"NEXUS6",
            "value":"XT1100"
        },
        1:{
            "column":"model_name",
            "key":"넥서스6",
            "value":"XT1100"
        },
        2:{
            "column":"mintit_key",
            "key":"NEXUS7",
            "value":"ME571KL"
        },
        3:{
            "column":"model_name",
            "key":"넥서스7",
            "value":"ME571KL"
        },
        4:{
            "column":"model_name",
            "key":"넥서스4",
            "value":"LG-E960"
        },
        5:{
            "column":"model_name",
            "key":"아이폰3GS",
            "value":"A1325"
        },
        6:{
            "column":"model_name",
            "key":"G6 플러스",
            "value":"LGM-G600P"
        },
        7:{
            "column":"model_name",
            "key":"Q6+",
            "value":"LGM-X600P"
        },
    },
    "joongabi":{
        
    },
    "bunjang":{
        0:{
            "column":"name",
            "key":"LG G7+ ThinQ",
            "value":"LGM-X600P"
        },
        
    },
}

In [5]:
for i in range(len(aka_table["mintit"])):
    print(aka_table["mintit"][i]["column"],\
          aka_table["mintit"][i]["key"],\
          aka_table["mintit"][i]["value"])


mintit_key NEXUS6 XT1100
model_name 넥서스6 XT1100
mintit_key NEXUS7 ME571KL
model_name 넥서스7 ME571KL
model_name 넥서스4 LG-E960
model_name 아이폰3GS A1325
model_name G6 플러스 LGM-G600P
model_name Q6+ LGM-X600P


# 민팃부분

In [6]:
mintit_index = pd.read_sql_table('mintit_index',db_class.engine_conn)
mintit_index.insert(8,"pattern",None,True)
mintit_index.insert(9,"storage",None,True)
pattern_list = []
storage_list = []

for i, row in mintit_index.iterrows():
    mintit_row = row.copy()
    
    #모델코드 패턴매칭부분
    
    is_aka_matched = 0
    aka_value = ""
    for i in range(len(aka_table["mintit"])):
        if (row[aka_table["mintit"][i]["column"]].lower().replace(" ",""))\
        .find(aka_table["mintit"][i]["key"].lower().replace(" ",""))!=-1:
            print(aka_table["mintit"][i]["column"],\
              aka_table["mintit"][i]["key"],\
              aka_table["mintit"][i]["value"])
            aka_value = aka_table["mintit"][i]["value"]
            is_aka_matched = 1
    
    if is_aka_matched == 1:
        pattern_list.append(aka_value)
    elif row['brand_name'] == '삼성전자':
        pattern_list.append(re.search("S(M|HW|HV)-[a-zA-Z][0-9]+",row['mintit_key']).group())
        
    elif row['brand_name'] == 'LG전자' :
        if re.search("L(GM|M|G)-[a-zA-Z]+[0-9]+",row['mintit_key']) is not None:
            pattern_list.append(re.search("L(GM|M|G)-[a-zA-Z]+[0-9]+",row['mintit_key']).group())
        else:
            pattern_list.append("엘지기타")
        
    elif row['brand_name'] == '애플':
        
        is_apple_find = 0
        apple_model_code = ""
        for i, row in phone_list.iterrows():
            mintit_name = re.sub(' [0-9]+G','',mintit_row['model_name'])
            
            if mintit_name.lower().replace(" ","")==row['pl_name'].lower().replace(" ",""):
                is_apple_find = 1
                apple_model_code = row['pl_model_code']
        if is_apple_find == 1:
            pattern_list.append(apple_model_code)
            
        else:
            
            is_apple_find = 0
            apple_model_code = ""
            mintit_key = ""
            for i, row in phone_list.iterrows():
                mintit_key = re.search('(?<=AIP)([0-9]+( )?(PRO( MAX)?|PM|Plus|MINI|S|C)?( )?(Plus|P)?( )?|X(R|S)?( )?(MAX|M)\
                                         ?|( )?SE)',mintit_row['mintit_key']).group()
                
                
                if mintit_key.lower().replace(" ","")==(xstr(row['pl_info_version']) + xstr(row['pl_info_subname_1']) +\
                                                          xstr(row['pl_info_subname_2'])).lower().replace(" ","")\
                and row['pl_info_model'] == "iphone":
                    is_apple_find = 1
                    apple_model_code = row['pl_model_code']
            
            if is_apple_find == 1:
                pattern_list.append(apple_model_code)
            else:
                pattern_list.append("애플기타")
            
    else:
        pattern_list.append(None)
        
        
    #용량부분
    if re.search("(?<=-)[0-9]+(G|T)",mintit_row['mintit_key']) is not None:
        storage_list.append(re.search("(?<=-)[0-9]+(G|T)",mintit_row['mintit_key']).group())
        
    elif pattern_list[-1] in list(phone_list['pl_model_code']):
        guess_single = "".join(list(phone_list[phone_list['pl_model_code'] == pattern_list[-1]]['pl_storage'])).split("|")
        if len(guess_single) == 1:
            storage_list.append(guess_single[0])
        else:
            storage_list.append(None)
    
    else:
        storage_list.append(None)
        
mintit_index["pattern"] = pattern_list
for i in range(len(storage_list)):
    if storage_list[i] and storage_list[i].find("G")!=-1:
        storage_list[i] = storage_list[i].replace("G","")
    elif storage_list[i] and storage_list[i].find("T")!=-1:
        storage_list[i] = int(storage_list[i].replace("T",""))*1024
mintit_index["storage"] = storage_list


model_name G6 플러스 LGM-G600P
mintit_key NEXUS6 XT1100
model_name 넥서스6 XT1100
mintit_key NEXUS7 ME571KL
model_name 넥서스7 ME571KL
model_name 넥서스4 LG-E960
model_name Q6+ LGM-X600P
model_name 아이폰3GS A1325
model_name 아이폰3GS A1325
model_name 아이폰3GS A1325


In [7]:
mintit_index

,id,brand_id,brand_name,generation,model_name,mintit_key,retail_price,model_id,pattern,storage
0,1,1207,삼성전자,갤럭시 S,갤럭시 S21,SM-G991,999900,427016,SM-G991,256
1,2,1207,삼성전자,갤럭시 S,갤럭시 S21 울트라 256G,SM-G998-256G,1452000,427018,SM-G998,256
2,3,1207,삼성전자,갤럭시 S,갤럭시 S21 플러스,SM-G996,1199000,427017,SM-G996,256
3,4,1207,삼성전자,갤럭시 S,갤럭시 S21 울트라 512G,SM-G998-512G,1599400,427019,SM-G998,512
4,5,1207,삼성전자,갤럭시 S,갤럭시 S20 FE,SM-G781,0,421403,SM-G781,128
5,6,1207,삼성전자,갤럭시 S,갤럭시S20 플러스,SM-G986,1353000,420113,SM-G986,256
6,7,1207,삼성전자,갤럭시 S,갤럭시S20,SM-G981,1248500,420112,SM-G981,128
7,8,1207,삼성전자,갤럭시 S,갤럭시S20 울트라,SM-G988,1595000,420114,SM-G988,256
8,9,1207,삼성전자,갤럭시 S,갤럭시 S10 5G 512G,SM-G977-512G,1556500,419791,SM-G977,512
9,10,1207,삼성전자,갤럭시 S,갤럭시 S10+ 512G,SM-G975-512G,1397000,419793,SM-G975,512


# 출고가크롤러부분

In [8]:
price_index = pd.read_sql_table('price_list',db_class.engine_conn)
price_index.insert(5,"pattern",None,True)
pattern_list = []
for i, row in price_index.iterrows():
    #모델코드매칭부분
    text = ""
    if row['model_code'] != None :
        text = regex.sub("(?<=[0-9]+)[a-zA-Z]+.*","",row['model_code'])
    if len(text) < 3:
        text = None
    pattern_list.append(text)
    
    #용량부분 => 이미되어있음
    
price_index["pattern"] = pattern_list

In [9]:
price_index

,price_id,model_name,model_code,price,storage,pattern
0,1,,LG-LV8500,236500,None,LG-LV8500
1,2,G 스타일로,None,297000,None,None
2,3,G 플렉스,LG-F340L,799700,None,LG-F340
3,4,G3 비트,LG-F470L,352000,None,LG-F470
4,5,G3 스크린,LG-F490L,154000,None,LG-F490
5,6,G3A,None,451000,None,None
6,7,G6,LGM-G600L,899800,64GB,LGM-G600
7,8,G6,LGM-G600LR,819500,32GB,LGM-G600
8,9,G7 ThinQ,LM-G710N,898700,64GB,LM-G710
9,10,G7 플러스 ThinQ,LM-G710N,976800,128GB,LM-G710


# 중가비 부분

In [10]:
joongabi_index = pd.read_sql_table('joongabi_index',db_class.engine_conn)
joongabi_index.insert(6,"pattern",None,True)
joongabi_index.insert(7,"storage",None,True)
pattern_list = []
storage_list = []
for i,row in joongabi_index.iterrows():
    joongabi_row = row.copy()
    
    #모델코드 매칭부분
    if row['brand_name'] == "apple":
        
        is_apple_find = 0
        apple_model_code = ""
        for i, row in phone_list.iterrows():
            joongabi_name = re.sub(' [0-9]+G(B)?(_리퍼)?','',joongabi_row['model_name'])
            joongabi_name = joongabi_name.replace("플러스","Plus").replace("+","Plus")
            if joongabi_name.lower().replace(" ","")==row['pl_name'].lower().replace(" ",""):
                is_apple_find = 1
                apple_model_code = row['pl_model_code']
                
        if is_apple_find == 1:
            pattern_list.append(apple_model_code)
            
        else:
            is_apple_find = 0
            apple_model_code = ""
            joongabi_key = ""
            for i, row in phone_list.iterrows():
                joongabi_key = re.search('(?<=AIP)([0-9]+(PM|Plus|M|S|C)?( )?(Plus|P)?( )?|X(R|S)?( )?(MAX|M)?|( )?SE([0-9]+)?)',\
                                          joongabi_row['model_code']).group()
                
                if joongabi_key.find("PM")!=-1:
                    joongabi_key = joongabi_key.replace("PM","PRO MAX")
                elif joongabi_key.find("MAX")!=-1:
                    pass
                elif joongabi_key.find("Plus")!=-1:
                    pass
                elif joongabi_key.find("P")!=-1:
                    joongabi_key = joongabi_key.replace("P","Plus")
                elif joongabi_key.find("M")!=-1:
                    joongabi_key = joongabi_key.replace("M","MINI")
                
                
                if joongabi_key.lower().replace(" ","")==(xstr(row['pl_info_version']) + xstr(row['pl_info_subname_1']) +\
                                                          xstr(row['pl_info_subname_2'])).lower().replace(" ",""):
                    is_apple_find = 1
                    apple_model_code = row['pl_model_code']
            
            if is_apple_find == 1:
                pattern_list.append(apple_model_code)
            else:
                pattern_list.append("애플기타")
        
    elif row['brand_name'] == "samsung":
        pattern_list.append(re.search("S(M|HW|HV)-[a-zA-Z][0-9]+",row['model_code']).group())
    elif row['brand_name'] == "lg":
        pattern_list.append(re.search("L(GM|M|G)-[a-zA-Z]+[0-9]+",row['model_code']).group())
    else:
        pattern_list.append(None)
        
    #용량부분
    if re.search("(?<=-)[0-9]+(G|T)",joongabi_row['model_code']) is not None:
        storage_list.append(re.search("(?<=-)[0-9]+(G|T)",joongabi_row['model_code']).group())
        
    elif re.search("(?<=_)[0-9]+(G|T)",joongabi_row['model_code']) is not None:
        storage_list.append(re.search("(?<=_)[0-9]+(G|T)",joongabi_row['model_code']).group())
        
    elif re.search("(?<= )[0-9]+(G|T)(B)?",joongabi_row['model_name']) is not None:
        storage_list.append(re.search("(?<= )[0-9]+(G|T)(B)?",joongabi_row['model_name']).group())
    
    #단일용량 기종 거르는 부분
    elif pattern_list[-1] in list(phone_list['pl_model_code']):
        guess_single = "".join(list(phone_list[phone_list['pl_model_code'] == pattern_list[-1]]['pl_storage'])).split("|")
        if len(guess_single) == 1:
            storage_list.append(guess_single[0])
        else:
            storage_list.append(None)
    else:
        storage_list.append(None)
    
joongabi_index["pattern"] = pattern_list
for i in range(len(storage_list)):
    if storage_list[i] and storage_list[i].find("G")!=-1:
        storage_list[i] = storage_list[i].replace("GB","").replace("G","")
    elif storage_list[i] and storage_list[i].find("T")!=-1:
        storage_list[i] = int(storage_list[i].replace("TB","").replace("T",""))*1024
joongabi_index["storage"] = storage_list

In [11]:
joongabi_index

,id,brand_id,brand_name,model_code,model_id,model_name,pattern,storage
0,1,1,apple,AIP12PM-512GB,422,아이폰12Pro Max 512G,A2411,512
1,2,1,apple,AIP12PM-256GB,421,아이폰12Pro Max 256G,A2411,256
2,3,1,apple,AIP12PM-128GB,420,아이폰12Pro Max 128G,A2411,128
3,4,1,apple,AIP12P-512G,417,아이폰12Pro 512G,A2407,512
4,5,1,apple,AIP12P-256G,416,아이폰12Pro 256G,A2407,256
5,6,1,apple,AIP12P-128G,415,아이폰12Pro 128G,A2407,128
6,7,1,apple,AIP12-256G,414,아이폰12 256G,A2403,256
7,8,1,apple,AIP12-128G,413,아이폰12 128G,A2403,128
8,9,1,apple,AIP12-64G,412,아이폰12 64G,A2403,64
9,10,1,apple,AIP12M-256GB,425,아이폰12 MINI 256G,A2399,256


In [12]:
# AIP => iphone
# 그다음오는숫자 => 그대로 숫자보존
# - 나오기전영어 => P단독:PRO / PM: PRO MAX / M: MINI / SE인경우 일단패스 or model_name 참조로 우회
#

# 번장부분

In [13]:
bunjang_index = pd.read_sql_table('bunjang_index',db_class.engine_conn)
bunjang_index.insert(5,"pattern",None,True)
pattern_list = []
for i,row in bunjang_index.iterrows():
    bunjang__row = row.copy()
    #모델코드 매칭부분
    is_aka_matched = 0
    aka_value = ""
    for i in range(len(aka_table["bunjang"])):
        if (row[aka_table["bunjang"][i]["column"]].lower().replace(" ",""))\
        .find(aka_table["bunjang"][i]["key"].lower().replace(" ",""))!=-1:
            print(aka_table["bunjang"][i]["column"],\
              aka_table["bunjang"][i]["key"],\
              aka_table["bunjang"][i]["value"])
            aka_value = aka_table["bunjang"][i]["value"]
            is_aka_matched = 1
    
    if is_aka_matched == 1:
        pattern_list.append(aka_value)
    
    elif row['bj_code'][0] == "S":
        pattern_list.append(re.search("S(M|HW|HV)-[a-zA-Z][0-9]+",row['bj_code']).group())
    elif row['bj_code'][0] == "L":
        pattern_list.append(re.search("L(GM|M|G)-[a-zA-Z]+[0-9]+",row['bj_code']).group())
    elif row['bj_code'][0:2] == "iP":
        
        is_apple_find = 0
        apple_model_code = ""
        for i, row in phone_list.iterrows():
            bunjang_name = bunjang__row['name']
            bunjang_name = bunjang_name.replace("플러스","Plus").replace("+","Plus")
            if bunjang_name.lower().replace(" ","")==row['pl_name'].lower().replace(" ",""):
                is_apple_find = 1
                apple_model_code = row['pl_model_code']
        if is_apple_find == 1:
            pattern_list.append(apple_model_code)
        else:
            pattern_list.append("애플기타")
            
    else:
        pattern_list.append(None)
            
    #용량 => 이미되어있음
        
bunjang_index["pattern"] = pattern_list

name LG G7+ ThinQ LGM-X600P


In [14]:
bunjang_index

,id,bj_code,name,storage,key,pattern
0,1,SM-N986N,갤럭시 노트20 울트라 5G,256,DGTLSSSMN986N256GB,SM-N986
1,2,SM-N981N,갤럭시 노트20 5G,256,DGTLSSSMN981N256GB,SM-N981
2,3,SM-N976K,갤럭시 노트10 플러스 5G,512,DGTLSSGN10P5G512GB,SM-N976
3,4,SM-N976K,갤럭시 노트10 플러스 5G,256,DGTLSSGN10P5G256GB,SM-N976
4,5,SM-N971K,갤럭시 노트10 5G,256,DGTLSSGN105G256GB,SM-N971
5,6,SM-N960K,갤럭시 노트9,512,DGTLSSGN9512GB,SM-N960
6,7,SM-N960K,갤럭시 노트9,128,DGTLSSGN9128GB,SM-N960
7,8,SM-N950K,갤럭시 노트8,256,DGTLSSGN8256GB,SM-N950
8,9,SM-N950K,갤럭시 노트8,64,DGTLSSGN864GB,SM-N950
9,10,SM-G998N,갤럭시 S21 울트라,512,DGTLSSSMG998N512GB,SM-G998


In [15]:
### => 번장 아이폰은 bj코드부분을 띄어쓰기 다없애고 모두 소문자화 == g5_phone_list 아이폰에서 pl_model_name 띄어쓰기 다날리고소문자화

In [23]:
join_keys = ['pattern','storage'] #모델명으로 데이터 조인
b_merge = bunjang_index.drop('id',axis=1).astype({'storage': 'float'})
j_merge = joongabi_index.drop('id',axis=1).fillna(0).astype({'storage': 'float'})
m_merge = mintit_index.fillna(0).astype({'storage': 'float'})
df_OUTER_JOIN = pd.merge(b_merge, j_merge, left_on=join_keys, right_on=join_keys, how='outer')
df_OUTER_JOIN = pd.merge(df_OUTER_JOIN,m_merge, left_on=join_keys, right_on=join_keys, how='outer')
# display(df_OUTER_JOIN)
result = df_OUTER_JOIN.drop(['name','brand_id_x','brand_name_x','model_id_x','model_name_x','id','generation','retail_price',
                            'brand_id_y','brand_name_y','model_id_y','model_name_y'],axis=1)
result = result[['pattern', 'storage', 'key','model_code','mintit_key']]
result = result.fillna(0)
result = result.astype({'storage': 'int'})
result = result.rename(columns={'key':'번장키','model_code':'중가비키','mintit_key':'민팃키'})
result = result.replace(0,np.NAN).drop_duplicates()
# display(result)
result_for_join = result.rename(columns={'pattern':'pl_model_code'})
result_for_join = pd.merge(result_for_join,phone_list, left_on='pl_model_code', right_on='pl_model_code', how='inner')
display(result_for_join[['pl_id','pl_model_name','pl_model_code','storage' ,'번장키', '중가비키','민팃키']].replace(0,np.NaN))
result_for_join = result_for_join[['pl_model_code','storage' ,'번장키', '중가비키','민팃키']].replace(0,np.NaN)
result_for_join.columns = ['original_code', 'price_code', 'mintit_code','bunjang_code','joongabi_code']

,pl_id,pl_model_name,pl_model_code,storage,번장키,중가비키,민팃키
0,220,GalaxyNote20 Ultra,SM-N986,256.0,DGTLSSSMN986N256GB,SM-N986,SM-N986
1,219,GalaxyNote20,SM-N981,256.0,DGTLSSSMN981N256GB,SM-N981,SM-N981
2,85,GalaxyNote10 Plus,SM-N976,512.0,DGTLSSGN10P5G512GB,SM-N976_512G,SM-N976-512G
3,85,GalaxyNote10 Plus,SM-N976,256.0,DGTLSSGN10P5G256GB,SM-N976_256G,SM-N976-256G
4,84,GalaxyNote10,SM-N971,256.0,DGTLSSGN105G256GB,SM-N971_256G,SM-N971-256G
5,86,GalaxyNote9,SM-N960,512.0,DGTLSSGN9512GB,SM-N960_512G,SM-N960-512G
6,86,GalaxyNote9,SM-N960,128.0,DGTLSSGN9128GB,SM-N960_128G,SM-N960-128G
7,87,GalaxyNote8,SM-N950,256.0,DGTLSSGN8256GB,SM-N950_256G,SM-N950-256G
8,87,GalaxyNote8,SM-N950,64.0,DGTLSSGN864GB,SM-N950_64G,SM-N950-64G
9,231,Galaxy S21 Ultra,SM-G998,512.0,DGTLSSSMG998N512GB,NaN,SM-G998-512G


In [25]:
import module.db_module as db_module #db연결 정의모듈 (id,pw 로컬환경따라 다름)

db_class = db_module.Database() #db연결 생성
result_for_join.to_csv("0729 코드매칭.csv",encoding="utf-8-sig")

result_for_join.to_sql(name='code_matching', con=db_class.engine, if_exists='append', index=False)


In [38]:
if 'A1688' in list(phone_list['pl_model_code']):
    print("godgod")

godgod


In [39]:
phone_list['pl_model_code']

0             A2221
1             A2215
2             A2218
3             A2097
4             A2101
5             A2105
6             A1901
7             A1905
8             A1897
9             A1778
10            A1784
11            A1723
12            A1688
13            A1687
14            A1586
15            A1524
16            A1530
17            A1532
18            A1428
19            A1387
20            A1332
21          SM-A908
22          SM-A505
23          SM-A405
24          SM-A305
25          SM-A205
26          SM-G887
27          SM-G885
28          SM-A920
29          SM-A530
30          SM-A600
31          SM-A750
32          SM-A520
33          SM-A810
34          SM-A310
35          SM-A510
36          SM-A710
37          SM-A800
38          SM-A500
39          SM-A300
40          SM-A700
41         SHW-M100
42          SM-J250
43          SM-J330
44          SM-J530
45          SM-J730
46          SM-J320
47          SM-J710
48          SM-J510
49          SM-J700


# 번장 텍스트마이닝

In [40]:
bunjang_index_2 = pd.read_sql_table('bunjang_index',db_class.engine_conn)
bunjang_index_2.insert(5,"pattern",None,True)
pattern_list = []
for i,row in bunjang_index_2.iterrows():
    bunjang__row = row.copy()
    #모델코드 매칭부분
    if row['bj_code'][0] == "S":
        pattern_list.append(re.search("S(M|HW|HV)-[a-zA-Z][0-9]+",row['bj_code']).group())
    elif row['bj_code'][0] == "L":
        pattern_list.append(re.search("L(GM|M|G)-[a-zA-Z]+[0-9]+",row['bj_code']).group())
    elif row['bj_code'][0:2] == "iP":
        
        is_apple_find = 0
        apple_model_code = ""
        for i, row in phone_list.iterrows():
            bunjang_name = bunjang__row['name']
            bunjang_name = bunjang_name.replace("플러스","Plus").replace("+","Plus")
            if bunjang_name.lower().replace(" ","")==row['pl_name'].lower().replace(" ",""):
                is_apple_find = 1
                apple_model_code = row['pl_model_code']
        if is_apple_find == 1:
            pattern_list.append(apple_model_code)
        else:
            
            #이부분에 추가중##이부분에 추가중##이부분에 추가중##이부분에 추가중#
            bunjang_key = bunjang__row['key']
            if bunjang_key[0:6] == "DGTLAP" :
                bunjang_key = re.sub("(512|256|128|64|32|16)GB","",bunjang_key)
                bunjang_key = re.sub("DGTLAP(IPHONE|AIP|IP)","",bunjang_key)
                
            
            is_apple_find = 0
            apple_model_code = ""
            for i, row in phone_list.iterrows():
                if bunjang_key== xstr(row['pl_info_version']) + xstr(row['pl_info_subname_1']) + xstr(row['pl_info_subname_2']) :
                    is_apple_find = 1
                    apple_model_code = row['pl_model_code']
                
            if is_apple_find == 1:
                pattern_list.append(apple_model_code)
                
            else:
                pattern_list.append("애플기타")
                
           
    
            #이부분에 추가중##이부분에 추가중##이부분에 추가중##이부분에 추가중#

    #용량 => 이미되어있음
    

        
bunjang_index_2["pattern"] = pattern_list
bunjang_index_2

,id,bj_code,name,storage,key,pattern
0,1,SM-N986N,갤럭시 노트20 울트라 5G,256,DGTLSSSMN986N256GB,SM-N986
1,2,SM-N981N,갤럭시 노트20 5G,256,DGTLSSSMN981N256GB,SM-N981
2,3,SM-N976K,갤럭시 노트10 플러스 5G,512,DGTLSSGN10P5G512GB,SM-N976
3,4,SM-N976K,갤럭시 노트10 플러스 5G,256,DGTLSSGN10P5G256GB,SM-N976
4,5,SM-N971K,갤럭시 노트10 5G,256,DGTLSSGN105G256GB,SM-N971
5,6,SM-N960K,갤럭시 노트9,512,DGTLSSGN9512GB,SM-N960
6,7,SM-N960K,갤럭시 노트9,128,DGTLSSGN9128GB,SM-N960
7,8,SM-N950K,갤럭시 노트8,256,DGTLSSGN8256GB,SM-N950
8,9,SM-N950K,갤럭시 노트8,64,DGTLSSGN864GB,SM-N950
9,10,SM-G998N,갤럭시 S21 울트라,512,DGTLSSSMG998N512GB,SM-G998


In [18]:
"DGTL"[0:4]

'DGTL'

In [24]:
str(None)

'None'

In [25]:
def xstr(s):
    if s is None:
        return ''
    return str(s)